In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 4
CTOI = {65: 0, 67: 1, 71: 2, 84: 3}
C_TO_I = lambda x: CTOI[x]
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = False

  def add(self, k) :
    if not k :
      self.output = True
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:])

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie):
    self.trie = trie
    self.trie.fail = self.trie
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output += v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for c in s :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      cnt += u.output
    return cnt

def sol() :
  N, M = map(int, input().split())
  trie = Trie()
  
  dna = input().rstrip()
  marker = input().rstrip()
  mutations = {marker} #naive (O(M(M+1) // 2) == O(M^2)) => 많아봤자 1만
  for i in range(1, M+1) : #diff 
    for j in range(M-i) : #start
      mutations.add(marker[:j] + marker[j:j+i+1][::-1]+ marker[j+i+1:])
  
  for m in mutations :
    trie.add(m)
  ac = AhoCorasick(trie)
  ans = 0
  ans += ac.query(dna)
  return ans

for _ in range(int(input())) :
  ans = []
  ans.append(sol())
  sys.stdout.write('\n'.join(map(str, ans)) + '\n')

### (거의)자력솔
- 돌연변이는, 길이 $2 \to |M|$ 인 구간의 연속하는 문자열을 뒤집는 것을 만드는 것으로 구현할 수 있다.
  - 모든 돌연변이의 조합은 $O(M^2)$ 이다. M의 길이가 100이므로, 충분히 가능하다.
- 돌연변이를 만들어서, 그것이 $S$에 포함되는지 확인하면 된다.
- 매칭 개수를 반환하는 방법을 몰라서 풀이를 참고했지만, 기본적인 아이디어와 풀이는 맞았다.
- 추가로, DNS 구조의 성분은 A, G, T, C로 4개밖에 없으므로 이걸 반영하면 더 빠르게 풀 수 있다.